In [2]:
import numpy as np
import pandas as pd

In [240]:
rawdata = pd.read_csv('links.txt', sep=',', names = ['row', 'column', 'weight'])

rows = set()
for j in rawdata.row.unique():
    rows.add(j)
cols = set()
for i in rawdata.column.unique():
    cols.add(i)

temp_df = pd.DataFrame()



for missing_row in cols.difference(rows):
    temp_df = temp_df.append([[missing_row,missing_row,0]], ignore_index=True)
    
for missing_col in rows.difference(cols):
    temp_df = temp_df.append([[missing_col,missing_col,0]], ignore_index=True)
temp_df.columns = ['row', 'column', 'weight']

trans_df = rawdata.append(temp_df, ignore_index=True)

trans_df.row = trans_df.row.astype(str)
trans_df.column = trans_df.column.astype(str)


data = trans_df.set_index(['row','column']).unstack('column')

Z = np.array(data.weight,dtype=float)

# replace nan with 0
Z[np.isnan(Z)] = 0

# row should represent cited 
# col should represent citing
Z = Z.transpose()

np.fill_diagonal(Z, 0)

col_sum = Z.sum(axis = 0) 

H = np.copy(Z)

# dangling node
d = []

# calculate dangling nodes and do normalization
for col in range(H.shape[1]):
    # dangling node
        
    if col_sum[col] == 0:
        d.append(1)
        continue
    
    d.append(0)
    # non-dangling node
    H[:,col] /= col_sum[col]

D =  np.array([d])  
        
#temp_df.columns = ['row', 'column', 'weight']
#temp_df

In [241]:
P = H.copy()
pi = np.array([[1/H.shape[0] for i in range(H.shape[0])]]).T

alpha = 0.85
epsilon = 0.00001

# normalized artcile 
pi = np.array([[1/H.shape[0] for i in range(H.shape[0])]]).T
A = np.array([[1/H.shape[0] for i in range(H.shape[0])]]).T

# start iteartion
iteration = 1
while True:
    new_pi = alpha * (H + A.dot(D)).dot(pi) + (1- alpha) * A
    e = np.linalg.norm(new_pi-pi, ord=1)
    pi = new_pi
    
    print ("iteration {0}: {1}".format(iteration, e))
    iteration += 1
    if e < epsilon:
        break

pi

iteration 1: 0.7855686429333226
iteration 2: 0.17165512912916014
iteration 3: 0.08176422023597503
iteration 4: 0.04849724938583972
iteration 5: 0.03114176053935483
iteration 6: 0.020621094721404467
iteration 7: 0.013960505720150796
iteration 8: 0.009701041898520215
iteration 9: 0.0068554030987561234
iteration 10: 0.0048950244819863506
iteration 11: 0.0035260762666439248
iteration 12: 0.0025593120101086416
iteration 13: 0.0018695665866134638
iteration 14: 0.0013733731939511327
iteration 15: 0.001014101287543642
iteration 16: 0.0007527555202661328
iteration 17: 0.0005612270336468218
iteration 18: 0.0004200490521078405
iteration 19: 0.00031556090689202864
iteration 20: 0.0002378514572574088
iteration 21: 0.00017979610470704628
iteration 22: 0.00013626607601955744
iteration 23: 0.00010349075269642533
iteration 24: 7.875090935276082e-05
iteration 25: 6.002636361380311e-05
iteration 26: 4.5818386723973266e-05
iteration 27: 3.501550911529295e-05
iteration 28: 2.6788703319299496e-05
iteration 

array([[4.38729652e-05],
       [2.74427015e-05],
       [1.73087650e-04],
       ...,
       [2.39789158e-05],
       [1.90948271e-05],
       [2.75902329e-05]])

In [235]:

ef = (100 * H.dot(pi) / pi.sum()).T[0].tolist()
ef.sort(reverse=True)
ef[:20]


[1.4336036254994982,
 1.3985584051954771,
 1.2226553354236727,
 0.6726914368510128,
 0.6582147727511158,
 0.6282736460119049,
 0.5714471377774861,
 0.47599571676862695,
 0.4729837427569814,
 0.4353271668217957,
 0.42541052300199556,
 0.3823354203046117,
 0.3812600510010095,
 0.3757729471869965,
 0.36905239732555406,
 0.32699549248121007,
 0.3242251543290151,
 0.3160714829587204,
 0.3136038335900688,
 0.3081371933243322]

In [3]:
case_df = pd.read_csv('case.txt', header = None, sep=',', dtype = float)

Z = np.array(case_df)


np.fill_diagonal(Z, 0)

col_sum = Z.sum(axis = 0) 

H = np.copy(Z)

H

array([[0., 0., 2., 0., 4., 3.],
       [3., 0., 1., 1., 0., 0.],
       [2., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 1.],
       [8., 0., 3., 0., 0., 2.],
       [0., 0., 0., 0., 0., 0.]])

In [11]:
# dangling node
d = []

# calculate dangling nodes and do normalization
for col in range(H.shape[1]):
    # dangling node
        
    if col_sum[col] == 0:
        d.append(1)
        continue
    
    d.append(0)
    # non-dangling node
#    for row in range(H.shape[0]):
#        H[row, col] = H[row, col]/col_sum(col)

from sklearn.preprocessing import normalize
H= normalize(H,norm='l1',axis = 0)

D =  np.array([d])  

H

array([[0.        , 0.        , 0.28571429, 0.        , 0.8       ,
        0.5       ],
       [0.23076923, 0.        , 0.14285714, 1.        , 0.        ,
        0.        ],
       [0.15384615, 0.        , 0.        , 0.        , 0.2       ,
        0.        ],
       [0.        , 0.        , 0.14285714, 0.        , 0.        ,
        0.16666667],
       [0.61538462, 0.        , 0.42857143, 0.        , 0.        ,
        0.33333333],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

In [6]:
P = H.copy()
pi = np.array([[1/H.shape[0] for i in range(H.shape[0])]]).T

pi

array([[0.16666667],
       [0.16666667],
       [0.16666667],
       [0.16666667],
       [0.16666667],
       [0.16666667]])

In [7]:
alpha = 0.85
epsilon = 0.00001

# normalized artcile 
pi = np.array([[1/H.shape[0] for i in range(H.shape[0])]]).T
A = np.array([[3/14, 2/14, 5/14, 1/14, 2/14, 1/14]]).T

pi
A

array([[0.21428571],
       [0.14285714],
       [0.35714286],
       [0.07142857],
       [0.14285714],
       [0.07142857]])

In [14]:
# start iteartion
iteration = 1
while True:
    b = float(alpha * D.dot(pi) + (1 - alpha))
    
    new_pi = alpha * H.dot(pi) + np.dot(b,A)
    e = np.linalg.norm(new_pi-pi, ord=1)
    pi = new_pi
#    new_pi = alpha * (H + A.dot(D)).dot(pi) + (1- alpha) * A
#    e = np.linalg.norm(new_pi-pi, ord=1)
#    pi = new_pi
    
    print ("iteration {0}: {1}".format(iteration, e))
    iteration += 1
    if e < epsilon:
        break

pi

iteration 1: 2.896774479455022e-06


array([[0.30402213],
       [0.16360283],
       [0.18979629],
       [0.04661905],
       [0.27531241],
       [0.02064729]])

In [12]:
tm = H.dot(pi)

ef = normalize(tm,norm='l1',axis = 0)

ef


#ef = 100 * H.dot(pi) / pi.sum()).T[0].tolist()



#ef.sort(reverse=True)
#ef[:20]

array([[0.34051006],
       [0.17203742],
       [0.12175455],
       [0.03653164],
       [0.32916632],
       [0.        ]])